In [47]:
import codecs
from pythainlp.tokenize import word_tokenize
from pythainlp.tag import pos_tag
from nltk.tokenize import RegexpTokenizer
import glob
import nltk
import re
import os
import pickle
import dill

In [48]:
#จัดการประโยคซ้ำ
data_not=[]
def Unique(p):
    text=re.sub("<^[>]*>","",p)
    text=re.sub("\[(.*?)\]","",text)
    text=re.sub("\[\/(.*?)\]","",text)
    if text not in data_not:
        data_not.append(text)
        return True
    else:
        return False

# อ่านข้อมูลจากไฟล์
def get_data(fileopen):
    """
    สำหรับใช้อ่านทั้งหมดทั้งในไฟล์ทีละรรทัดออกมาเป็น list
    """
    lines = []
    str_line = ""
    with open(fileopen, 'r',encoding='utf-8-sig') as f:
        for line in f.readlines():
            
            if(line == "\n"):
                lines.append(str_line)
                lines[-1] = lines[-1].replace("\n"," ")
                str_line = ""
            else:
                str_line += line
        lines.append(str_line.replace("\n"," "))
        lines = "\n".join(lines)
        return lines
#     return [a for a in lines if Unique(a)] # เอาไม่ซ้ำกัน

def getall(lista):
    ll=[]
    for i in lista:
        o=True
        for j in ll:
            if re.sub("\[(.*?)\]","",i)==re.sub("\[(.*?)\]","",j):
                o=False
                break
        if o==True:
            ll.append(i)
    return ll
    

# ใช้สำหรับกำกับ pos tag เพื่อใช้กับ NER
# print(text2conll2002(t,pos=False))
def postag(text):
    listtxt=[i for i in text.split('<newline>') if i!='']
    list_word=[]
    for data in listtxt:
        list_word.append(data.split('\t')[0])
    #print(text)
    list_word=pos_tag(list_word,engine="perceptron")
    text=""
    i=0
    for data in listtxt:
        text+=data.split('\t')[0]+'\t'+list_word[i][1]+'\t'+data.split('\t')[1]+'<newline>'  #เราจะไปเดินเล่นที่\tO\nหนองคาย\tB-location\n
        i+=1
    return text


# จัดการกับ tag ที่ไม่ได้ tag
def toolner_to_tag(text):
    text=text.strip()
    text=re.sub("<[^>]*>","",text)
    text=re.sub("(\[\/(.*?)\])","\\1***",text)#.replace('(\[(.*?)\])','***\\1')#  ตัดการกับพวกไม่มี tag word
    text=re.sub("(\[\w+\])","***\\1",text)
    text2=[]
    for i in text.split('***'):
        if "[" in i:
            text2.append(i)
        else:
            text2.append("[word]"+i+"[/word]")
    text="".join(text2)#re.sub("[word][/word]","","".join(text2))
    return text.replace("[word][/word]","")

# เตรียมตัวตัด tag ด้วย re
pattern = r'\[(.*?)\](.*?)\[\/(.*?)\]'
tokenizer = RegexpTokenizer(pattern) # ใช้ nltk.tokenize.RegexpTokenizer เพื่อตัด [TIME]8.00[/TIME] ให้เป็น ('TIME','ไง','TIME')

# แปลง text ให้เป็น conll2002
def text2conll2002(text,pos=True):
    """
    ใช้แปลงข้อความให้กลายเป็น conll2002
    """
    text=toolner_to_tag(text) # นำไปใส่ tag [word]
    text=text.replace("''",'"')
    text=text.replace("’",'"').replace("‘",'"')#.replace('"',"")
    tag=tokenizer.tokenize(text) # แยก tag ออกมาจากข้อความ
    j=0
    conll2002="" # ประกาศตัวแปรเก็บ conll2002
    for tagopen,text,tagclose in tag: # ลูปใน tag โดยเป็น (tagopen,text,tagclose)
        word_cut=word_tokenize(text,engine="newmm") # ใช้ตัวตัดคำ newmm ของ PyThaiNLP
        i=0
        txt5=""
        while i<len(word_cut): #ลูปตามจำนวน token ที่ตัดในtag
            if word_cut[i]=="''" or word_cut[i]=='"':pass
            elif i==0 and tagopen!='word': # ไม่เป็น tag [word] และเป็น i หรือตัวเริ่มต้น tag
                txt5+=word_cut[i]
                txt5+='\t'+'B-'+tagopen  
            elif tagopen!='word':
                txt5+=word_cut[i]
                txt5+='\t'+'I-'+tagopen 
            else: # เป็น [word]
                txt5+=word_cut[i]
                txt5+='\t'+'O'
            txt5+='<newline>' #ลาว\t-location\n
            #j+=1
            i+=1
        conll2002+=txt5
    if pos==False:
        return conll2002
    return postag(conll2002) # เพิ่ม postag ใส่

def alldata_list(lists, head=False, pos=True):
    get_tile = 0 if head == True else 1
    data_all=[]
    for data in lists:
        data = data.split("\n")[get_tile:]
        data = " ".join(data)
        data_num=[]
        txt=text2conll2002(data,pos=pos).split('<newline>') # นำไปแปลงเป็น conll2002
        for d in txt:
            tt=d.split('\t')
            if d!="":
                if len(tt)==3:
                    data_num.append((tt[0],tt[1],tt[2])) 
                else:
                    data_num.append((tt[0],tt[1]))
        #print(data_num)
        data_all.append(data_num)
    return data_all

In [49]:
path = "../dataset/LAW/"
data1  = []
for name_folder in os.listdir(path):
    for f_name in os.listdir(path+name_folder):
        data1.append(get_data(path+name_folder+"/"+f_name))

datatofile_ = alldata_list(data1, head=False, pos=False)
datatofile_pos = alldata_list(data1, head=False, pos=True) # นำไปผ่านขั้นตอน 1 2 3 4

In [50]:
def clear_empty_list(datatofile):
    temp_data = []
    for data in datatofile:
        if data != []:
            temp_data.append(data)
    return temp_data

In [51]:
len(datatofile_pos)

101

# save data

In [52]:
path = "../dataset/CoNLL2002-dataset/"
file_name="blog"

with open(path+file_name+"-pos.conll","w", encoding="utf-8-sig") as f:
    i=0
    while i<len(datatofile_pos):
        for j in datatofile_pos[i]:
            f.write(j[0]+"\t"+j[1]+"\t"+j[2]+"\n")
        if i+1<len(datatofile_pos):
            f.write("\n")
        i+=1

with open(path+file_name+".conll","w",  encoding="utf-8-sig") as f:
    i=0
    while i<len(datatofile_):
        for j in datatofile_[i]:
            f.write(j[0]+"\t"+j[1]+"\n")
        if i+1<len(datatofile_):
            f.write("\n")
        i+=1

In [53]:
path = "../dataset/data/"

pickle.dump(datatofile_pos, open(path+"blog-pos.data", 'wb'))
pickle.dump(datatofile_, open(path+"blog.data", 'wb'))

# load data

In [54]:
path = "../dataset/data/"

with open(path+"blog-pos.data", 'rb') as file:
    datatofile = dill.load(file)

len(datatofile)

101